In [20]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import json
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain



In [21]:

load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')

In [22]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

cache_store = LocalFileStore("./cache/")

In [23]:
loader = DirectoryLoader('../data', glob="*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()
# pretty_print_docs(docs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 725.91it/s]


In [24]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,
    chunk_overlap=50,
)
chunked = text_splitter.split_documents(docs)
# print(len(chunked))
# pretty_print_docs(chunked)

In [25]:
cache_store = LocalFileStore("./nomic_cache/")

In [26]:
embeddings_model = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [27]:
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model, cache_store, namespace=embeddings_model.model)

In [28]:
%%time
db = FAISS.from_documents(chunked, cached_embedder)

CPU times: user 228 ms, sys: 67.3 ms, total: 295 ms
Wall time: 4.2 s


In [72]:
llm = Ollama(model="mistral", temperature=0)
query = "How do I defeat Dreki?"

In [30]:
retriever = db.as_retriever()
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

Document 1:

Straight out of the gate, the Dreki will spit a ball of saliva toward you – you’ll be dealt damage if you do not move in time. And while it’s arguably easier to just dodge this attack, it’s also possible to block it with your shield, or even parry it back into the creature if you’re using the Dauntless Shield, dealing damage and temporarily stunning it.
Keep in mind, though, to parry the attack, you will need to be very precise with your timing, as it’ll only work if you block the attack just before it hits you. Once the Dreki is stunned, we recommend rushing the creature and unleashing as much heavy damage as possible before the stun effect runs out. This includes using your Runic Attack. Using the Runic Attack early in a fight is generally recommended so you can get the cooldown timer going as soon as possible.
Fortunately, the Dreki does not have any unblockable attacks until it reaches half health, so technically speaking, every attack the Dreki throws your way can be 

In [31]:
%%time
print(llm.invoke(query))

To defeat Dreki, the fire-breathing dragon in Elden Ring, you'll need to follow these steps:

1. Reach Dragonburrow: To find Dreki, head to Dragonburrow, which is located northwest of Limgrave. It's a large cave with a campfire outside and a massive dragon skull at the entrance.

2. Stealth: As you enter the cave, be cautious because Dreki will wake up if he spots you. To avoid being detected, try to sneak past him by sticking close to the wall and moving slowly.

3. Avoid his attacks: When Dreki does spot you, move quickly to avoid his fire breath attack. He'll also swing his tail and smash the ground with it, so watch out for those as well.

4. Chip away at his health: If you can get behind Dreki, you can deal some serious damage with your attacks. Keep hitting him until you start to chip away at his massive health bar.

5. Break his wings: Eventually, you'll want to break one of Dreki's wings by dodging and weaving around his attacks while striking the wing repeatedly. This will mak

In [32]:
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# compressed_docs = compression_retriever.get_relevant_documents(query)
# pretty_print_docs(compressed_docs)

In [33]:
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)

In [34]:
%%time
# c1 = chain({"query": query})

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.15 µs


In [35]:
# print(c1['result'])

In [36]:
# print(c1["source_documents"])

In [37]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

Document 1:

In turn, the Dreki’s saliva attack will be upgraded into an unblockable attack as it’ll now become electrified, leaving a nasty trail of electricity in its wake.

Due to these unblockable attacks, we found it was best to keep a distance during the second phase of the fight, as it was much easier to get caught off guard by the electricity attacks. Instead, we only approached within melee distance while the Dreki was temporarily stunned or knocked back, and only unleashing a few heavy melee attacks before retreating to a safe distance.

Once you have managed to deplete the Dreki’s health bar, press R3 when prompted to finally defeat the creature with the help of Atreus. Defeating the Dreki will reward you with a Chaos Flame, +4 Dragon Tooths, and a Light Runic Attack (Flames of Anguish).
After collecting all the loot in the area, continue west and squeeze through the gap in the wall. Before continuing to the main objective to the south, let’s take a quick detour and follow t

In [38]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)


CPU times: user 334 µs, sys: 14 µs, total: 348 µs
Wall time: 349 µs


In [39]:
%%time
c2 = chain({"query": query})

CPU times: user 60.6 ms, sys: 29.1 ms, total: 89.7 ms
Wall time: 16.7 s


In [40]:
print(c2["result"])

During the fight with Dreki, it's best to keep your distance during the second phase due to its electrified saliva attack that becomes unblockable. Instead, approach within melee distance only while the Dreki is temporarily stunned or knocked back and unleash a few heavy melee attacks before retreating to a safe distance. The Dreki's initial saliva attack can be blocked with your shield or parried back into the creature if using the Dauntless Shield, but its melee attacks should still be avoided or blocked. Once it reaches half health, the Dreki will have unblockable attacks, making every attack potentially dangerous to block. To deal significant damage, rush the creature and unleash heavy attacks during its stun effect, including using your Runic Attack early in the fight. Remember to conserve your health by looking for resources beside crates in the area. With these strategies, you can defeat Dreki, earning a Chaos Flame, +4 Dragon Tooths, and a Light Runic Attack (Flames of Anguish)

In [56]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=Cohere(temperature=1), retriever=compression_retriever, chain_type="refine"
)
c3 = chain({"query": query})

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: {'message': "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.


CPU times: user 65.3 ms, sys: 11.9 ms, total: 77.2 ms
Wall time: 22.7 s


In [57]:
print(c3["result"])

 To defeat the Dreki you must avoid its electrifies attack, by keeping your distance whilst also damaging the monster. The Dreki has five health bars, so you’ll need to deal quite a bit of damage to defeat the monster. Considering the context of the passage overall, barracks slashes and Atreus’s arrows are the most effective ways to damage the Dreki.

You should also be aware that the Dreki will try to spit saliva at you, and although this can merely be dodged, it is also possible to block it with your shield, or even parry it back into the creature if you’re using the Dauntless Shield, dealing damage and temporarily stunning it. Whilst the Dreki is stunned, you should rush the creature and unleashes as much heavy damage as possible before the stun effect runs out. 

In addition to this saliva attack, the Dreki will also perform two major melee attacks that deal significant damage should you not block them. The first melee attack will see the Dreki jumping toward you, and landing sever

In [75]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="stuff"
)
c4 = chain({"query": query})

CPU times: user 59.9 ms, sys: 42 ms, total: 102 ms
Wall time: 16.1 s


In [76]:
print(c4["result"])

 To defeat Dreki, you should keep your distance during the second phase of the fight due to its unblockable electrified saliva attacks. Approach only when the Dreki is temporarily stunned or knocked back, and unleash a few heavy melee attacks before retreating to a safe distance. When prompted, press R3 to finally defeat the creature with Atreus' help. Remember that the Dreki has five sectioned health bars, so dealing significant damage is necessary to defeat it. You can block or parry its saliva attack, but be precise with your timing for a successful parry. The Dreki does not have any unblockable attacks until it reaches half health. Additionally, you can collect resources and Hacksilver along the way as you follow the path northwest from The Forge.


In [45]:
print(c3["result"])

To defeat Dreki, you should dodge its initial saliva spit if possible or block/parry it. In the second phase, keep a distance during the fight as its saliva attack has been upgraded into an unblockable electrified attack that leaves a nasty trail of electricity in its wake. Approach within melee distance only while the Dreki is temporarily stunned or knocked back and unleash a few heavy melee attacks before retreating to a safe distance due to these unblockable attacks being easier to get caught off guard by the electricity attacks. After depleting Dreki's health bar, press R3 when prompted to finally defeat the creature with the help of Atreus.

To summarize, follow Atreus north through a small hole in the rock wall and prepare to battle a larger Dreki with five sectioned health bars. Parrying Dreki's initial saliva spit requires precise timing by blocking just before the attack hits, as its major melee attacks include a tail swipe that can be blocked or dodged, and a chomp after leap